## Mistral v0.3: Multi-Function Calling & Dependencies! 🚀

Mistral just dropped version 0.3, and it natively supports function calling! 🛠️

I tested multi-functions & dependencies. It didn't work initially, but I found a solution that works like a charm. Check out my video and try it on Colab. Enjoy!

Btw, @enginerrprompt (https://x.com/enginerrprompt) suggested a better name which is "nested call" for dependencies. Thanks!

Watch video here: https://youtu.be/MZlRaclJ4YI

Follow me on X: https://x.com/unclecode

In [ ]:
%%capture
!pip install mistral_inference

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

'/root/mistral_models/7B-Instruct-v0.3'

In [ ]:
import json
from mistral_common.protocol.instruct.tool_calls import Function, Tool
from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage, SystemMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")
model = Transformer.from_folder(mistral_models_path)

completion_request = ChatCompletionRequest(
    tools=[
        Tool(
            function=Function(
                name="get_current_weather",
                description="Get the current weather",
                parameters={
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "format": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                    },
                    "required": ["location", "format"],
                },
            )
        ),
        Tool(
            function=Function(
                name="get_current_time",
                description="Get the current time",
                parameters={
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        }
                    },
                    "required": ["location"],
                },
            )
        )
    ],
    messages=[
        UserMessage(content="What's the weather like today in Paris? Also tell me the time in San Francisco?"),
    ],
)

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

json.loads(result)


In [ ]:
tokenizer.encode_chat_completion(completion_request).text

'<s>[AVAILABLE_TOOLS]▁[{"type":▁"function",▁"function":▁{"name":▁"get_current_weather",▁"description":▁"Get▁the▁current▁weather",▁"parameters":▁{"type":▁"object",▁"properties":▁{"location":▁{"type":▁"string",▁"description":▁"The▁city▁and▁state,▁e.g.▁San▁Francisco,▁CA"},▁"format":▁{"type":▁"string",▁"enum":▁["celsius",▁"fahrenheit"],▁"description":▁"The▁temperature▁unit▁to▁use.▁Infer▁this▁from▁the▁users▁location."}},▁"required":▁["location",▁"format"]}}},▁{"type":▁"function",▁"function":▁{"name":▁"get_current_time",▁"description":▁"Get▁the▁current▁time",▁"parameters":▁{"type":▁"object",▁"properties":▁{"location":▁{"type":▁"string",▁"description":▁"The▁city▁and▁state,▁e.g.▁San▁Francisco,▁CA"}},▁"required":▁["location"]}}}][/AVAILABLE_TOOLS][INST]▁You▁are▁a▁helpful▁assistant.▁And▁this▁is▁a▁test.<0x0A><0x0A>What\'s▁the▁weather▁like▁today▁in▁Paris?▁Also▁tell▁me▁the▁time▁in▁San▁Francisco?[/INST]'

In [ ]:
completion_request = ChatCompletionRequest(
    tools=[
        Tool(
            function=Function(
                name="get_current_temperature",
                description="Get the current temperature in celsius",
                parameters={
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        }
                    },
                    "required": ["location"],
                },
            )
        ),
        Tool(
            function=Function(
                name="convert_temperature",
                description="Convert temperature",
                parameters={
                    "type": "object",
                    "properties": {
                        "temperature": {
                            "type": "number",
                            "description": "The temperature to convert",
                        },
                        "from_unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                        "to_unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                    },
                    "required": ["temperature", "from_unit", "to_unit"],
                },
            )
        )
    ],
    messages=[
        UserMessage(content="What's the weather like today in Paris? Also convert it to fahrenheit?"),
    ],
)

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=256, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)


[{"name": "get_current_temperature", "arguments": {"location": "Paris"}}], [{"name": "convert_temperature", "arguments": {"temperature": "$(get_current_temperature.result)", "from_unit": "celsius", "to_unit": "fahrenheit"}}]


## Okay, it seems it can understand, but running it multiple times yields different responses. Let’s add a bit of a system message to get more consistent responses. 🤓

Also, I noticed that sometimes the JSON response is not proper, so I’ve added that into the system message as well.

In [ ]:
completion_request = ChatCompletionRequest(
    tools=[
        Tool(
            function=Function(
                name="get_current_temperature",
                description="Get the current temperature in celsius",
                parameters={
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        }
                    },
                    "required": ["location"],
                },
            )
        ),
        Tool(
            function=Function(
                name="convert_temperature",
                description="Convert temperature",
                parameters={
                    "type": "object",
                    "properties": {
                        "temperature": {
                            "type": "number",
                            "description": "The temperature to convert",
                        },
                        "from_unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                        "to_unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                    },
                    "required": ["temperature", "from_unit", "to_unit"],
                },
            )
        )
    ],
    messages=[
        SystemMessage(content="You are a helpful assistant. Your job is to select tools relevant to the user query. In the case of multiple tools, if the tools are dependent on each other, and one tool's input parameters come from another function, use @ followed by the function name for the parameter value. Remember you response type is List[Dict<String, Any>]."),
        UserMessage(content="What's the weather like today in Paris? Also convert it to fahrenheit?"),
    ],
)

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=128, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

json.loads(result)


[{'name': 'get_current_temperature',
  'arguments': {'location': 'Paris, France'}},
 {'name': 'convert_temperature',
  'arguments': {'temperature': '@get_current_temperature.result',
   'from_unit': 'celsius',
   'to_unit': 'fahrenheit'}}]

## 😎 More Complexity: I've noticed OpenAI doesn't perform well when more than two functions are needed. Let's test it with additional functions.

In [ ]:
tools=[
        Tool(
            function=Function(
                name="get_current_temperature",
                description="Get the current temperature in celsius",
                parameters={
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        }
                    },
                    "required": ["location"],
                },
            )
        ),
        Tool(
            function=Function(
                name="convert_temperature",
                description="Convert temperature",
                parameters={
                    "type": "object",
                    "properties": {
                        "temperature": {
                            "type": "number",
                            "description": "The temperature to convert",
                        },
                        "from_unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                        "to_unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                    },
                    "required": ["temperature", "from_unit", "to_unit"],
                },
            )
        ),
        Tool(
            function=Function(
                name="adjust_air_conditioner",
                description="Adjust the air conditioner based on the temperature",
                parameters={
                    "type": "object",
                    "properties": {
                        "temperature": {
                            "type": "number",
                            "description": "The temperature in fahrenheit to set the air conditioner",
                        },
                    },
                    "required": ["temperature"],
                },
            )
        ),
        Tool(
            function=Function(
                name="email_temperature",
                description="Email the temperature to the home owner",
                parameters={
                    "type": "object",
                    "properties": {
                        "email": {
                            "type": "string",
                            "description": "The email address to send the temperature to",
                        },
                        "temperature": {
                            "type": "number",
                            "description": "The temperature in fahrenheit to email",
                        },
                    },
                    "required": ["email", "temperature"],
                },
            )
        ),
    ]

completion_request = ChatCompletionRequest(
    tools=tools,
    messages=[
        SystemMessage(content="You are a helpful assistant. Your job is to select tools relevant to the user query. In the case of multiple tools, if the tools are dependent on each other, and one tool's input parameters come from another function, use @ followed by the function name for the parameter value. Remember you response type is List[Dict<String, Any>]."),
        UserMessage(content="What's the weather like today in Paris? Also convert it to fahrenheit, email this converted  temperature to the home owner (tom@unclecode.com), then use it to adjust the air conditioner."),
    ],
)

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=256, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

json.loads(result)


[{'name': 'get_current_temperature',
  'arguments': {'location': 'Paris, France'}},
 {'name': 'convert_temperature',
  'arguments': {'temperature': '@get_current_temperature.result',
   'from_unit': 'celsius',
   'to_unit': 'fahrenheit'}},
 {'name': 'email_temperature',
  'arguments': {'email': 'tom@unclecode.com',
   'temperature': '@convert_temperature.result'}},
 {'name': 'adjust_air_conditioner',
  'arguments': {'temperature': '@convert_temperature.result'}}]

## Nicely done! Let's really push hard and see if we can break it! 🤔 This time, I challenge its context-awareness and reasoning.

In [ ]:
completion_request = ChatCompletionRequest(
    tools=tools,
    messages=[
        SystemMessage(content="You are a helpful assistant. Your job is to select tools relevant to the user query. In the case of multiple tools, if the tools are dependent on each other, and one tool's input parameters come from another function, use @ followed by the function name for the parameter value. Remember you response type is List[Dict<String, Any>]."),
        UserMessage(content="What's the weather like today in Paris? Also convert it to fahrenheit, email the initial temperature in celsius to the home owner (tom@unclecode.com), then use the fahrenheit to adjust the air conditioner."),
    ],
)

tokens = tokenizer.encode_chat_completion(completion_request).tokens
out_tokens, _ = generate([tokens], model, max_tokens=256, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

json.loads(result)

[{'name': 'get_current_temperature', 'arguments': {'location': 'Paris'}},
 {'name': 'convert_temperature',
  'arguments': {'temperature': '@get_current_temperature.result',
   'from_unit': 'celsius',
   'to_unit': 'fahrenheit'}},
 {'name': 'email_temperature',
  'arguments': {'email': 'tom@unclecode.com',
   'temperature': '@get_current_temperature.result'}},
 {'name': 'adjust_air_conditioner',
  'arguments': {'temperature': '@convert_temperature.result'}}]

## Again, Nicely done! Okay I wanna go for a crazy one. 😈

In [ ]:
tools = [
        Tool(
            function=Function(
                name="sin",
                description="Calculate the sine of a number",
                parameters={
                    "type": "object",
                    "properties": {
                        "value": {
                            "type": "number",
                            "description": "The value to calculate the sine of",
                        }
                    },
                    "required": ["value"],
                },
            )
        ),
        Tool(
            function=Function(
                name="cos",
                description="Calculate the cosine of a number",
                parameters={
                    "type": "object",
                    "properties": {
                        "value": {
                            "type": "number",
                            "description": "The value to calculate the cosine of",
                        }
                    },
                    "required": ["value"],
                },
            )
        ),
        Tool(
            function=Function(
                name="exp",
                description="Calculate the exponential of a number",
                parameters={
                    "type": "object",
                    "properties": {
                        "value": {
                            "type": "number",
                            "description": "The value to calculate the exponential of",
                        }
                    },
                    "required": ["value"],
                },
            )
        ),
        Tool(
            function=Function(
                name="pow",
                description="Calculate the power of a number",
                parameters={
                    "type": "object",
                    "properties": {
                        "base": {
                            "type": "number",
                            "description": "The base value",
                        },
                        "exponent": {
                            "type": "number",
                            "description": "The exponent value",
                        },
                    },
                    "required": ["base", "exponent"],
                },
            )
        ),
    ]
completion_request = ChatCompletionRequest(
    tools=tools,
    messages=[
        SystemMessage(content="You are a helpful assistant. Your job is to select tools relevant to the user query. In the case of multiple tools, if the tools are dependent on each other, and one tool's input parameters come from another function, use @ followed by the function name for the parameter value. Remember you response type is List[Dict<String, Any>]."),
        UserMessage(content="Calculate the result of pow( (sin(exp(10)) + cos(pow(2, 3))) , 4) + exp(10). Make sure break it done properly an duse all available tools. Response only JSON, make sure the order matches with the mathematical priority order."),
    ],
)

tokens = tokenizer.encode_chat_completion(completion_request).tokens
out_tokens, _ = generate([tokens], model, max_tokens=256, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

# json.loads(result)
print(result)


[
  {
    "name": "exp",
    "arguments": {"value": 10}
  },
  {
    "name": "sin",
    "arguments": {"value": "@result_of_exp"}
  },
  {
    "name": "cos",
    "arguments": {"value": "@result_of_pow_2_3"}
  },
  {
    "name": "pow",
    "arguments": {"base": "@result_of_sin", "exponent": 4}
  },
  {
    "name": "pow",
    "arguments": {"base": "@result_of_sum", "exponent": 4}
  },
  {
    "name": "pow",
    "arguments": {"base": 2, "exponent": 3}
  },
  {
    "name": "add",
    "arguments": {"value1": "@result_of_sin", "value2": "@result_of_cos"}
  },
  {
    "name": "exp",
    "arguments": {"value": 10


## Well, honestly not bad. This definitely needs a bit more fine-tuning. I'm trying to come up with more weird and tricky examples, then generate a synthetic dataset and fine-tune 😅. Please stay tuned.

## One more thing, before we end, let's try OpenAI as well.

In [ ]:
%%capture
!pip install openai

In [ ]:
from openai import OpenAI
from google.colab import userdata
dict_tools = [t.model_dump() for t in tools]
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

messages=[
        # {'role': 'system','content': "You are a helpful assistant. Your job is to select tools relevant to the user query. In the case of multiple tools, if the tools are dependent on each other, and one tool's input parameters come from another function, use @ followed by the function name for the parameter value. Remember you response type is List[Dict<String, Any>]."},
        {'role': 'user','content':"Calculate the result of pow( (sin(exp(10)) + cos(pow(2, 3))) , 4) + exp(10). Make sure break it done properly an duse all available tools. Response only JSON, make sure the order matches with the mathematical priority order."}
]

In [ ]:
response = client.chat.completions.create(
    model = "gpt-4o",
    messages = messages,
    tool_choice="required",
    tools=dict_tools
)

In [ ]:
response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Vs5qjLoUSVsabga3LGjx2iH9', function=Function(arguments='{"value": 10}', name='exp'), type='function'), ChatCompletionMessageToolCall(id='call_Gr2usuJMUgaeg8KQ3kucdLk4', function=Function(arguments='{"base": 2, "exponent": 3}', name='pow'), type='function')])

In [ ]:
response = client.chat.completions.create(
    model = "gpt-4o",
    messages = messages,
    tools=dict_tools
)

In [ ]:
response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_u3JEHnqTxAZZBAdt9EixtcRc', function=Function(arguments='{"value":10}', name='exp'), type='function')])

## 🤨🤨🤨

## Well, I know OpenAI doesn't perform well when you need more than two tools. One way to handle this is to wrap everything into one function. Anyway, I keep liking Mistral v0.3 more and more. Let's see how it turns out after fine-tuning.